# Bokeh plotting template

Do all cleaning beforehand!

**Load in clean df**

In [4]:
# imports
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
from math import pi
import matplotlib.pyplot as plt

from bokeh.plotting import figure, show, output_file, output_notebook
from bokeh.models import ColumnDataSource, Range1d, HoverTool, Legend
from bokeh.models.widgets import Tabs, Panel
from bokeh.io import show
from bokeh.palettes import BuGn, Category20b, Spectral, Plasma, Viridis, YlOrRd, PuOr, BuPu, BuGn, brewer, GnBu, PRGn, Inferno256, PuRd, RdPu, PiYG, RdYlGn, YlGnBu
from bokeh.io import reset_output, output_notebook
reset_output()
output_notebook()

from bokeh.plotting import figure, output_file, save
from IPython.display import IFrame
from IPython.core.display import display, HTML
import tempfile


Loading BokehJS ...

In [5]:
df = pd.read_csv('../data/hatecrime_df_clean.csv')
df.head()

,Date of Offense,Time of Offense,Date Offense Reported,Report Year,Month,CCN,District,Block Location,Type of Hate Bias,Targeted Group,Top Offense Type,Address,gcode,lat,long,zipcodes
0,2012-01-08,1500-1505,2012-01-08,2012,1,12003845,3,1600 B/O 17th St NW,Sexual Orientation,Not Reported,Threats,1600 17th St NW Washington DC,"1600, 17th Street Northwest, Dupont Circle, Wa...",38.911250,-77.038588,20009
1,2012-01-14,0240-0250,2012-01-14,2012,1,12006716,3,1800 B/O 14th St NW,Sexual Orientation,Not Reported,Simple Assault,1800 14th St NW Washington DC,"1800, 14th Street Northwest, Cardozo/Shaw, Was...",38.914217,-77.032054,20009
2,2012-01-22,145,2012-01-22,2012,1,12010626,7,1300 B/O Alabama Ave SE,Sexual Orientation,Not Reported,ADW,1300 Alabama Ave SE Washington DC,"1300, Alabama Avenue Southeast, Washington, Di...",38.845056,-76.987412,20020
3,2012-01-22,1930,2012-01-22,2012,1,12010912,1,600 B/O H St NW,Race,Asian,Threats,600 H St NW Washington DC,"600, H Street Northwest, Chinatown, Washington...",38.899729,-77.020065,20001
4,2012-02-06,1900,2012-02-06,2012,2,12018396,3,1300 B/O Park Rd NW,Sexual Orientation,Not Reported,Simple Assault,1300 Park Rd NW Washington DC,"1300, Park Road Northwest, Columbia Heights, W...",38.931198,-77.029808,20010


# Horizontal bar examples

### Step 1: Create a function that creates a seperate dataframe for every categorical column that we want to analyze

In [6]:
# function to make a dataframe to count the unique values for each category in a given column
def prep_bokeh_df(df, column):
    count_df = pd.Series(df[column].value_counts()).reset_index(name='Hate crime count').rename(columns={'index':column})
    return count_df


Use the function to create multiple dfs for the categories we want to explore.

In [7]:
# make the new dfs
type_bias_bar_df = prep_bokeh_df(df, 'Type of Hate Bias')
targeted_group_bar_df = prep_bokeh_df(df, 'Targeted Group')
offense_type_bar_df = prep_bokeh_df(df, 'Top Offense Type')

### Step 2: Create a function to plot Bokeh categorical data into horizontal bars

In [8]:
# function to plot bokeh categorical data in a horizontal bar
# you input the dataframe, column, colorpalette, height, and width
def bokeh_hbar(df, column, title, colorpalette, height, width):
    TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom,tap"
    p = figure(plot_height=height,
        plot_width=width,
        title=title,
        tools=TOOLS,
        toolbar_location='right',
        y_range=df[column].unique()[::-1])
    p.hbar(y=df[column], right=df['Hate crime count'], 
    height=0.75,color=colorpalette[df[column].nunique()]) #color='#8968CD')
    p.yaxis.axis_label = column
    p.xaxis.axis_label = 'Number of hate crimes'
    p.select_one(HoverTool).tooltips = [
        (column, '@y'),
        ('Number of hate crimes', '@right')]
    return p

We display the number of hate crimes per type of hate bias, targeted group, and top offense type in tabs.

In [9]:
# update colorschemes
PuRd_20 = {20: PuRd[9][:-1]+ PuRd[9][:-1] + PuRd[9][:-5]}
PuRd_28 = {28: PuRd[9][:-1]+ PuRd[9][:-1] + PuRd[9][:-1] + PuRd[9][:-5]}


# intialize plots to be put in tabs
p_type = bokeh_hbar(type_bias_bar_df, 'Type of Hate Bias', "Number of hate crimes by type of hate bias", 
             PuRd, 450, 650)
p_target = bokeh_hbar(targeted_group_bar_df, 'Targeted Group', "Number of hate crimes by targeted group", 
            PuRd_28, 450, 650)
p_offense = bokeh_hbar(offense_type_bar_df, 'Top Offense Type', "Number of hate crimes by offense type", 
            PuRd_20, 450, 650)            

# format in tabs
tabs = Tabs(tabs=[
                 Panel(child=p_type, title='Type'),
                 Panel(child=p_target, title='Targeted group'),
                 Panel(child=p_offense, title='Offense type')])

show(tabs)

# Trending examples

### Step 1: Prepare dataframes for the time intervals we want to trend by.

In [10]:
# prep a dataframe to have one row per date with the count of crimes, and also include year, month
df_daily_counts = df['Date of Offense'].value_counts()
df_daily_counts = pd.Series(df_daily_counts).reset_index(name='Hate crime count').rename(columns={'index':'Date of Offense'})
df_daily_counts['Year'] = pd.to_datetime(df_daily_counts['Date of Offense']).dt.year
df_daily_counts['Month'] = pd.to_datetime(df_daily_counts['Date of Offense']).dt.month
df_daily_counts['MonthYear'] = pd.to_datetime(df_daily_counts['Date of Offense']).dt.to_period('M')# month_temp_df.head()

# df just for month and year trending
df_yearly_counts = df_daily_counts.groupby('Year').sum().reset_index()
df_monthly_counts = df_daily_counts.groupby('MonthYear').sum().reset_index()

### Step 2: Create function to plot trending data in vbars.

In [11]:
# create function to take in a df, column, title, and colormap to make a trending vbar
def bokeh_vbar(df, xcolumn, ycolumn, title, colormap, height, width):
    TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom,tap"
    p = figure(plot_height=height, plot_width=width,
            title=title,
            tools=TOOLS,
            toolbar_location='right')

    p.vbar(x=df[xcolumn], top=df[ycolumn], width=0.75, 
         color=colormap)

    p.xaxis.axis_label = xcolumn
    p.yaxis.axis_label = ycolumn
    p.select_one(HoverTool).tooltips = [
        (xcolumn, '@x'),
        ('Number of hate crimes', '@top'),
    ]

    return p


Show basic trending vbar by year.

In [12]:
# show yearly hate crime counts
p = bokeh_vbar(df_yearly_counts, 'Year', 'Hate crime count', 'Number of hate crimes per year',PuRd[8][:-1]+PuRd[9][:-6], 300, 600)
show(p)

### Step 3: Create function to trend data in a line.

In [13]:
# create function for line graph
def bokeh_line(df, xcolumn, ycolumn, title, line_color, line_width, xaxis_label, height, width):
    TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom,tap"
    p = figure(plot_height=height, plot_width=width,
        title=title,
        tools=TOOLS, x_axis_type="datetime",
        toolbar_location='right')

    p.line(x=df[xcolumn], y=df[ycolumn],line_color=line_color,
            line_width = line_width)

    p.xaxis.axis_label = xaxis_label
    p.yaxis.axis_label = ycolumn
    p.select_one(HoverTool).tooltips = [
        ('Month', '@x{%B %Y}'),
        ('Number of hate crimes', '@y')]
    p.select_one(HoverTool).formatters={ '@x': 'datetime'}

    return p

Show trending based on month.

In [14]:
# show monthly trending
p = bokeh_line(df_monthly_counts, 'MonthYear', 'Hate crime count', 'Number of hate crimes per month',
            "#872657", 2.5, 'Date', 300, 700)
show(p)

# Trending stacked bars

### Step 1: create a function to prepare a new df to store one row per timeframe with the category counts in each column, and a list of the unique values for that column

In [15]:
# function for all preproccesing for stacked bar trending graph
def bokeh_vbar_stack_prep(df, time_column, cat_column, tot_column):
    # prep the df to be one row per year with the category counts in each column
    stacked_df = df.groupby([time_column, cat_column]).size().unstack().reset_index()
    # find each category value
    cat_values = df[cat_column].unique()
    # add total count for hover tool
    stacked_df[tot_column] = stacked_df.drop(time_column, axis=1).sum(axis=1)
    # replace NaNs with 0s
    stacked_df = stacked_df.fillna(0)

    # return the prepares df and the unique values for the vbar
    return stacked_df, cat_values

### Step 2: create a function to plot the stacked bar trending graph

In [16]:
# function to plot the vbar_stack
def bokeh_vbar_stack(stacked_df, cat_values, time_column, title, xaxis_label, yaxis_label, height, width):
    TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom,tap"
    source = ColumnDataSource(data=stacked_df)
    p = figure(plot_width=width, plot_height=height, title=title,
        toolbar_location='above', tools=TOOLS, tooltips="$name @{Report Year}: @$name")

    colors = brewer['PuRd'][len(cat_values)]

    p.vbar_stack(cat_values, x=time_column, width=0.9,  source=source, color=colors,
                legend_label=cat_values.tolist())

    p.xaxis.axis_label = xaxis_label
    p.yaxis.axis_label = yaxis_label
    p.legend.location = "top_left"
    p.legend.orientation = "vertical"
    p.legend.click_policy='hide'

    p.add_layout(p.legend[0], 'right')

    return p

Plot the stacked bar.

In [17]:
# use the initial function to prep the df and category values
type_stacked_year, bias_types = bokeh_vbar_stack_prep(df, 'Report Year', 'Type of Hate Bias', 'Total hate crimes')

# plot the stacked bar
p = bokeh_vbar_stack(type_stacked_year, bias_types, 'Report Year', 'Hate crime counts per type over year',
                        'Year', 'Total hate crimes', 450, 1000)
show(p)

# Plot % change based on last time period

### Step 1: create dfs that store the counts per each time frame we want to analyze, and then store the percent change from the last period.

We prepare the dataframes to be used in Bokeh plotting easily, by breaking them down into Year and Month. We also apply pct_change() to identify the percentage change from the previous year. The dataframe is sorted in ascending order so that it generates the change between the previous and current year values.

In [18]:
df_daily_counts = df['Date of Offense'].value_counts()
df_daily_counts = pd.Series(df_daily_counts).reset_index(name='Hate crime count').rename(columns={'index':'Date of Offense'})
df_daily_counts['Year'] = pd.to_datetime(df_daily_counts['Date of Offense']).dt.year
df_daily_counts['Month'] = pd.to_datetime(df_daily_counts['Date of Offense']).dt.month
df_daily_counts['MonthYear'] = pd.to_datetime(df_daily_counts['Date of Offense']).dt.to_period('M')# month_temp_df.head()

# df just for month and year trending
df_yearly_counts = df_daily_counts.groupby('Year').sum().reset_index()

# find percentage change based on previous year
df_yearly_counts['pct_change'] = df_yearly_counts['Hate crime count'].pct_change()
df_yearly_counts['pct_change'] = df_yearly_counts['pct_change']*100
df_yearly_counts['color'] = ""
df_yearly_counts['color'][df_yearly_counts['pct_change'] >= 0] = '#EE6363' # red
df_yearly_counts['color'][df_yearly_counts['pct_change'] < 0] = '#71C671' # green

### Step 2: create a function to compute bar plots in Bokeh specifically for showing the percentage change from the previous year. Positive changes are green while negative changes are red.

In [19]:
# function to compute bar plot of percentage change from last year's count
def bokeh_bar_pct_change(df, yearcolumn, title, height, width):
    TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom,tap"
    p = figure(plot_height=height, plot_width=width,
        title=title,
        tools=TOOLS,
        toolbar_location='right')
    p.vbar(x=df[yearcolumn], top=df['pct_change'], width=0.75, color=df['color'])
    p.xaxis.axis_label = 'Year'
    p.yaxis.axis_label = '% Change from last year'
    p.select_one(HoverTool).tooltips = [
        ('Year', '@x'),
        ('Percent change from previous year', '@top'+'%'),
    ]
    return p

Show the change of hate crime counts over year.

In [20]:
p = bokeh_bar_pct_change(df_yearly_counts, 'Year', '% change in total hate crime counts from previous year',
                     300,450)
show(p)